In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import warnings

In [2]:
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

In [3]:
def prise_de_glace(df, alert, pg, window_size, i):
    
    df["var_S03"] = df["S03"] - df["S03"].shift(1)
    df["var_S04"] = df["S04"] - df["S04"].shift(1)
    
    def rolling_mediane(serie, window):
        valeur = [serie.iloc[0]]
        for i in range(1,len(serie)):
            tableau = serie[max(0, i-window+1):i+1]
            valeur.append(np.median(tableau))
        return valeur

    def rolling_moyenne(serie, window):
        valeur = [serie.iloc[0]]
        for i in range(1,len(serie)):
            tableau = serie[max(0, i-window+1):i+1]
            valeur.append(np.mean(tableau))
        return valeur
    
    df["difference"] = df["S03"] - df["S04"]
    df["difference_future"] = rolling_moyenne(df["difference"], window_size)
    df["var_S03_future"] = rolling_moyenne(df["var_S03"], window_size)
    df["var_S03_future_expo"] = np.exp(df["var_S03_future"])
    df["var_S04_future"] = rolling_moyenne(df["var_S04"], window_size)
    df["var_S04_future_expo"] = np.exp(df["var_S04_future"])
    df["taux_relatif"] = df["var_S03_future_expo"] / df["var_S04_future_expo"]
    df["taux_relatif_future"] = rolling_moyenne(df["taux_relatif"], window_size)
    df["taux_relatif_difference_future"] = df["taux_relatif_future"]*df["difference_future"]
    
    if i < window_size:
        alert = False
        df.iloc[-1, df.columns.get_loc('normale?')] = 1
        return alert, df, pg
    else:
        normal_frame = df[df["normale?"] == 1]
        
        start_index = 0
        df1 = normal_frame.iloc[start_index:]
        mean_median_rolling = df1["taux_relatif_difference_future"].mean()
        std_median_rolling = df1["taux_relatif_difference_future"].std()
        n = df1["taux_relatif_difference_future"].count() 
        confidence_level = 0.99 # pourcentage d'erreur toléré
        z_score = stats.norm.ppf((1 + confidence_level) / 2)
        confidence_interval_lower = mean_median_rolling - z_score * std_median_rolling
        confidence_interval_upper = mean_median_rolling + z_score * std_median_rolling
        
        if(df.iloc[-1, df.columns.get_loc('taux_relatif_difference_future')] < confidence_interval_lower or 
        df.iloc[-1, df.columns.get_loc('taux_relatif_difference_future')] > confidence_interval_upper):
            if(df.iloc[-2, df.columns.get_loc('normale?')] == 1):
                alert = True
        else:
            alert = False
            df.iloc[-1, df.columns.get_loc('normale?')] = 1 
            if(df.iloc[-2, df.columns.get_loc('normale?')] == 0):
                print("fin de prise de glace le ", df.index[-1])
                pg.iloc[-1, pg.columns.get_loc('fin')] = df.index[-1]
        
        return alert, df, pg

In [4]:
file_path = 'dataset/Picard_Evreux152_Mai2023-2024_3.csv'
df = pd.read_csv(file_path, sep=';')
df = df.set_index('date')
df.index = pd.to_datetime(df.index, format='%d/%m/%Y %H:%M', errors='coerce')
df.sort_values(by=['date'], inplace=True)
df['S03'] = df['S03'].str.replace(',', '.')
df['S04'] = df['S04'].str.replace(',', '.')
df['S03'] = pd.to_numeric(df['S03'], errors='coerce')
df['S04'] = pd.to_numeric(df['S04'], errors='coerce')
df['S03'].fillna(df['S03'].median(), inplace=True)
df['S04'].fillna(df['S04'].median(), inplace=True)

In [5]:
alert = False
window_size = 288
serie1 = pd.DataFrame(columns=df.columns)
pg1 = pd.DataFrame(columns=["numero", "debut", "fin"])
serie1['normale?'] = None
serie1 = pd.concat([serie1, df.iloc[[1]]], ignore_index=False)
serie1.loc[df.index[1], 'normale?'] = 1
j = 0
cumul_alert = False
serie1.to_csv('serie1.csv', index=True)
pg1.to_csv('pg1.csv', index=True)
for i in range(2, len(df)):
    if alert:
        j += 1
        print("La ", j, "ième prise de glace a été détectée à la date ", serie1.index[-1])
        new_row = pd.DataFrame([[j, serie1.index[-1], serie1.index[-1]]], columns=["numero", "debut", "fin"])
        pg1 = pd.concat([pg1, new_row], ignore_index=True)
    serie1 = pd.concat([serie1, df.iloc[[i]]], ignore_index=False)
    serie1.loc[df.index[i], 'normale?'] = 0
    alert = False
    alert, serie1, pg1 = prise_de_glace(serie1, alert, pg1, window_size, i)
    #serie1.to_csv('serie1.csv', index=True)
    cumul_alert = cumul_alert or alert
pg1.to_csv('pg1.csv', index=True)

if not cumul_alert:
    print("Pas de prise de glace détectée")
else:
    print("\nFin de la détection de prise de glace")


La  1 ième prise de glace a été détectée à la date  2023-06-21 16:45:00
fin de prise de glace le  2023-06-21 19:05:00
La  2 ième prise de glace a été détectée à la date  2023-06-22 15:45:00
fin de prise de glace le  2023-06-22 20:15:00
La  3 ième prise de glace a été détectée à la date  2023-07-04 15:00:00
fin de prise de glace le  2023-07-04 15:25:00
La  4 ième prise de glace a été détectée à la date  2023-07-04 21:05:00
fin de prise de glace le  2023-07-04 21:15:00
La  5 ième prise de glace a été détectée à la date  2023-07-05 15:00:00
fin de prise de glace le  2023-07-05 15:20:00
La  6 ième prise de glace a été détectée à la date  2023-07-11 15:30:00
fin de prise de glace le  2023-07-11 16:10:00
La  7 ième prise de glace a été détectée à la date  2023-07-12 15:35:00
fin de prise de glace le  2023-07-12 15:45:00
La  8 ième prise de glace a été détectée à la date  2023-07-30 15:40:00
fin de prise de glace le  2023-07-30 16:05:00
La  9 ième prise de glace a été détectée à la date  2023